In [1]:
import importlib.util
import time

import tensorflow as tf
import numpy as np

from skimage.transform import rescale
from tqdm import trange
from IPython.display import HTML

#Import the vizdoom package as "vd" since it can't be installed normally on Windows

vd_location = 'C:/Anaconda3/envs/doom/Lib/site-packages/vizdoom/vizdoom.pyd'
vizdoom = importlib.util.spec_from_file_location('vizdoom',
                                                 vd_location)
vd = importlib.util.module_from_spec(vizdoom)
vizdoom.loader.exec_module(vd)


In [2]:
#Specify the game scenario and the screen format/resolution

game = vd.DoomGame()
game.set_screen_format(vd.ScreenFormat.GRAY8)
game.set_depth_buffer_enabled(True)
game.set_screen_resolution(vd.ScreenResolution.RES_160X120)
game.load_config('deadly_corridor.cfg')

down_sample_ratio = 1
width = int(game.get_screen_width()*down_sample_ratio)
height = int(game.get_screen_height()*down_sample_ratio)
channels = game.get_screen_channels() + int(game.is_depth_buffer_enabled())

#Specify the available actions in the scenario

available_actions = game.get_available_buttons()
actions = [list(ohe) for ohe in list(np.identity(len(available_actions)))]
num_actions = len(available_actions)

#Specify the Q-network learning parameters

frame_delay = 12
buffer_size = 75000
epochs = 800
steps_per_epoch = 2000
learning_rate = 0.0025
gamma = 0
start_epsilon = 1.0
end_epsilon = 0.1
batch_size = 100
load_model = False
save_model = True
model_dir = './checkpoints/deadly_corridor.ckpt'
num_ckpts = 20


In [3]:
#Create a buffer object that holds a set of training experiences (state-action-reward tuples)

class Buffer():
    def __init__(self, size=1000):
        self.buffer = list()
        self.length = len(self.buffer)
        self.size = size
        
#Add a new experience to the buffer (remove the oldest experience if the buffer is already full)
        
    def add_experience(self, experience):
        if self.length + 1 >= self.size:
            self.buffer[0:(self.length + 1) - self.size] = []
        
        self.buffer.append(experience)
        self.length = len(self.buffer)
            
#Return a batch of experience arrays randomly sampled from the buffer
            
    def sample_buffer(self, sample_size):
        sample = np.random.randint(self.length, size=sample_size)
        s1 = np.concatenate([self.buffer[idx][0] for idx in sample], axis=0)
        a = np.array([self.buffer[idx][1] for idx in sample])
        r = np.array([self.buffer[idx][2] for idx in sample])
        s2 = np.concatenate([self.buffer[idx][3] for idx in sample], axis=0)
        terminal = np.array([self.buffer[idx][4] for idx in sample], dtype=np.int32)
        
        return s1, a, r, s2, terminal

#Downsample and normalize an image array representing the game state at a given time stamp

def preprocess(image, down_sample_ratio=1):
    if float(down_sample_ratio) != 1.0:
        image = rescale(image=image, scale=down_sample_ratio, mode='reflect')
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)

    return image

#Test the agent using a currently training or previously trained model

def test_agent(model, num_episodes, load_model, depth, training=True, session=None, model_dir=None):
    if load_model == True:
        sess = tf.Session()
        print('Loading model from', model_dir)
        tf.train.Saver().restore(sess, model_dir)
        
#Require an existing session if a pretrained model isn't provided
        
    elif load_model == False:
        sess = session

    game.set_sound_enabled(True)
    episode_rewards = list()
    
#Avoid reinitializing the game if this was already done by the training process
    
    if training == False:
        game.init()

    for i in range(num_episodes):
        game.new_episode()
    
        while not game.is_episode_finished():
            state = game.get_state()
            
            if depth == False:
                depth_buffer = np.zeros(state.screen_buffer.shape)
            elif depth == True:
                depth_buffer = state.depth_buffer
                
            state_buffer = np.stack((state.screen_buffer,
                                     depth_buffer), axis=-1)
            state1 = preprocess(state_buffer, down_sample_ratio)
            action = model.choose_action(sess, state1)[0]
            reward = game.make_action(actions[action])
            
#Add a delay between each time step so that the episodes occur at normal speed

            time.sleep(0.02)
        
        episode_rewards.append(game.get_total_reward())
        print('Test Episode {} Reward: {}'.format(i + 1, game.get_total_reward()))
        time.sleep(1)
    
#Avoid ending the game so that the training process can continue
    
    if training == False:
        game.close()
    
    return np.mean(episode_rewards)


In [4]:
#Create a Q-network to estimate values and choose actions for a given state

class Q_network():
    def __init__(self, network_name, height, width, channels, learning_rate=0.001):
        self.learning_rate = learning_rate
        self.s_t = tf.placeholder(tf.float32,
                                  shape=[None, height, width, channels],
                                  name=network_name + '_state'
                                 )
        self.a_t = tf.placeholder(tf.int32,
                                  shape=[None],
                                  name=network_name + '_action'
                                 )
        self.Q_target = tf.placeholder(tf.float32,
                                       shape=[None, num_actions],
                                       name=network_name + '_Q_target'
                                      )

        self.input_layer = tf.reshape(self.s_t,
                                      [-1, height, width, channels],
                                      name=network_name + '_input_layer'
                                     )
        self.conv1 = tf.layers.conv2d(inputs=self.input_layer,
                                      filters=32,
                                      kernel_size=[8, 8],
                                      strides=[4, 4],
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      name=network_name + '_conv1_layer'
                                     )
        self.conv2 = tf.layers.conv2d(inputs=self.conv1,
                                      filters=64,
                                      kernel_size=[4, 4],
                                      strides=[2, 2],
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      name=network_name + '_conv2_layer'
                                     )
        self.flatten = tf.layers.flatten(self.conv2,
                                         name=network_name + '_flatten'
                                        )
        self.dense = tf.layers.dense(inputs=self.flatten,
                                      units=512,
                                      activation=tf.nn.relu,
                                      name=network_name + '_dense1_layer'
                                    )
        self.Q_values = tf.layers.dense(inputs=self.dense,
                                        units=len(actions),
                                        activation=None,
                                        name=network_name + '_output_layer'
                                       )        
    
        self.best_action = tf.argmax(self.Q_values, 1)
        self.loss = tf.losses.mean_squared_error(self.Q_values,
                                                 self.Q_target)
        self.adam = tf.train.AdamOptimizer(learning_rate=self.learning_rate,
                                           name=network_name + '_adam'
                                          )
        self.train = self.adam.minimize(self.loss)
        
    def update_lr(self):
        self.learning_rate = 0.98*self.learning_rate
        
        return self.learning_rate

    def calculate_loss(self, session, s, q):
        L, _ = session.run([self.loss, self.train],
                           feed_dict={self.s_t: s,
                                      self.Q_target: q})
    
        return L

#Return the array of Q-values and the best action associated with a given state

    def get_Q_values(self, session, s):
        Q = session.run(self.Q_values,
                        feed_dict={self.s_t: s})

        return Q
    
    def choose_action(self, session, s):
        a = session.run(self.best_action,
                        feed_dict={self.s_t: s})
    
        return a
    
#Create a list of variable update operations

def update_graph(variables):
    update_ops = list()
    
#Assign weight values from the network created first to the one created second
    
    for idx, variable in enumerate(variables[:len(variables)//2]):
        op = variable.assign(variables[idx + len(variables)//2].value())
        update_ops.append(op)
    
    return update_ops

#Update the target network parameters to match those of the online network

def update_target(ops, session):
    for op in update_ops:
        session.run(op)


In [5]:
#For each time step, collect the following data:
#The current game state
#The action that was taken taken
#The reward obtained from the chosen action
#The next game state (store the first game state if the previous action ends the episode)
#A variable indicating whether the episode is over yet


tf.reset_default_graph()

#Instantiate the target network before the online network so that it's updated correctly

target_net = Q_network(network_name='target',
                       learning_rate=learning_rate,
                       height=height,
                       width=width,
                       channels=channels)
DQN = Q_network(network_name='online',
                learning_rate=learning_rate,
                height=height,
                width=width,
                channels=channels)

exp_buffer = Buffer(size=buffer_size)
session = tf.Session()
saver = tf.train.Saver(max_to_keep=num_ckpts, reshape=True)
weights = tf.trainable_variables()

update_ops = update_graph(weights)

if load_model == True:
    print('Loading model from', model_dir)
    tf.train.Saver().restore(session, model_dir)
    
elif load_model == False:
    session.run(tf.global_variables_initializer())

game.set_sound_enabled(False)
game.init()

t = 0
epoch_rank = list()
epoch_rank_depth = list()

#Accumulate experiences in the buffer using an epsilon-greedy strategy with three training phases

for epoch in range(epochs):
    epoch_rewards = list()
    
    for step in trange(steps_per_epoch, leave=True):
        experience = list()
        game.new_episode()
        
        while not game.is_episode_finished():
            state = game.get_state()
            
#Substitute an array of zeros for the depth buffer if that setting is disabled

            if game.is_depth_buffer_enabled() == False:
                depth_buffer = np.zeros(state.screen_buffer.shape)
            else:
                depth_buffer = state.depth_buffer
            
            state1_buffer = np.stack((state.screen_buffer, depth_buffer), axis=-1)
            state1 = preprocess(state1_buffer, down_sample_ratio)
            
#Explore the environment by choosing random actions with 100% probability for the first phase of training

            if epoch < 0.15*epochs:
                action = np.random.randint(num_actions)
            
#Increase the probability of greedily choosing an action by a constant amount at each epoch in the second phase
            
            elif epoch < 0.45*epochs:
                epsilon = start_epsilon - (epoch + 1 - 0.2*epochs)*(start_epsilon-end_epsilon)/(0.7*epochs)
            
                if np.random.uniform(0, 1) <= epsilon:
                    action = np.random.randint(num_actions)
                else:
                    action = DQN.choose_action(session, state1)[0]

#Select a random action with 10% probability in the final phase of training
                
            else:
                if np.random.uniform(0, 1) <= end_epsilon:
                    action = np.random.randint(num_actions)
                else:
                    action = DQN.choose_action(session, state1)[0]

            reward = game.make_action(actions[action], frame_delay)
            done = game.is_episode_finished()
            
            if done == False:
                state = game.get_state()
                state2_buffer = np.stack((state.screen_buffer, state.depth_buffer), axis=-1)
                state2 = preprocess(state2_buffer, down_sample_ratio)
                
            elif done == True:
                state2 = state1
        
#Add the experience obtained from each time step to the buffer

            t += 1
            exp_buffer.add_experience((state1, action, reward, state2, done))
        
#Sample a minibatch from the buffer if there are enough experiences in the buffer

        if exp_buffer.length > batch_size:
            s1, a, r, s2, terminal = exp_buffer.sample_buffer(batch_size)
            
#Get the target values from the target Q-network
            
            target_Q = np.max(target_net.get_Q_values(session, s2), axis=1)
            
#Train the online Q-network by using a minibatch to update the action-value function
            
            Q2 = DQN.get_Q_values(session, s1)
            Q2[np.arange(batch_size), a] = r + gamma*(1 - terminal)*target_Q
            DQN.calculate_loss(session, s1, Q2)
            
        epoch_rewards.append(game.get_total_reward())
        
#Increase the discount factor at each epoch until it reaches 0.99
    
    if gamma < 0.99:
        gamma = 1-.98*(1-gamma)
    elif gamma >= 0.99:
        gamma = 0.99
        
#Decrease the learning rate at each epoch

    DQN.update_lr()
    target_net.update_lr()
    
    print('Epoch {} Mean Reward: {}'.format(epoch + 1, np.mean(epoch_rewards)))
    
#Update the target network every 10 epochs
    
    if (epoch + 1) % 10 == 0 and epoch > 0:
        update_target(update_ops, session)
        
#Save the model and test the agent for 10 episodes every 20 epochs
    
    if (epoch + 1) % 20 == 0 and epoch > 0:
        if save_model == True:
            checkpoint = model_dir + '-' + str(epoch + 1)
            print('Epoch {} Model saved to {}'.format(epoch + 1, model_dir))
            saver.save(session, model_dir, global_step=epoch + 1)
            
        update_target(update_ops, session)

#Test the agent both with and without the depth buffer given
        
        print('Epoch {} test with depth buffer:'.format(epoch + 1))
        test_reward_depth = test_agent(DQN, num_episodes=10,
                                       training=True,
                                       load_model=False,
                                       depth=True,
                                       session=session,
                                       model_dir=model_dir)
        print('Average Test Reward (with depth buffer:)', test_reward_depth)
        
        print('Epoch {} test without depth buffer:'.format(epoch + 1))
        test_reward = test_agent(DQN, num_episodes=10,
                                 training=True,
                                 load_model=False,
                                 depth=False,
                                 session=session,
                                 model_dir=model_dir)
        print('Average Test Reward (without depth buffer):', test_reward)
        
        epoch_rank_depth.append((test_reward_depth, epoch + 1))
        epoch_rank.append((test_reward, epoch + 1))
        
#Return a sorted list of epoch checkpoints based on average test episode reward
        
print(sorted(epoch_rank, reverse=True))
print('{} time steps experienced during training'.format(t))
game.close()
    

100%|██████████| 2000/2000 [01:28<00:00, 22.65it/s]


Epoch 1 Mean Reward: -71.7723299331665


100%|██████████| 2000/2000 [01:28<00:00, 22.62it/s]


Epoch 2 Mean Reward: -68.08960192108154


100%|██████████| 2000/2000 [01:28<00:00, 22.50it/s]


...

Epoch 798 Mean Reward: 102.64886380767823


100%|██████████| 2000/2000 [17:51<00:00,  1.87it/s]


Epoch 799 Mean Reward: 68.1441583404541


100%|██████████| 2000/2000 [16:41<00:00,  2.00it/s]


Epoch 800 Mean Reward: 86.59523091125489
Epoch 800 Model saved to ./checkpoints/deadly_corridor.ckpt
Epoch 800 test with depth buffer:
Test Episode 1 Reward: -66.40925598144531
Test Episode 2 Reward: -115.19439697265625
Test Episode 3 Reward: -66.40925598144531
Test Episode 4 Reward: -66.40925598144531
Test Episode 5 Reward: -66.40925598144531
Test Episode 6 Reward: -66.40925598144531
Test Episode 7 Reward: -66.40925598144531
Test Episode 8 Reward: -66.40925598144531
Test Episode 9 Reward: -66.40925598144531
Test Episode 10 Reward: -17.944000244140625
Average Test Reward (with depth buffer:) -66.44124450683594
Epoch 800 test without depth buffer:
Test Episode 1 Reward: -44.82696533203125
Test Episode 2 Reward: -44.82696533203125
Test Episode 3 Reward: -18.439163208007812
Test Episode 4 Reward: -44.82696533203125
Test Episode 5 Reward: -44.82696533203125
Test Episode 6 Reward: -31.977081298828125
Test Episode 7 Reward: 2.681060791015625
Test Episode 8 Reward: -44.82696533203125
Test Epi

In [6]:
#Get a list of checkpoints saved during training

ckpts = tf.train.get_checkpoint_state('checkpoints').all_model_checkpoint_paths

#Test the trained model from a certain checkpoint by only choosing actions with a greedy strategy

for i in range(len(ckpts)):
    test_reward = test_agent(DQN, num_episodes=20,
                             training=False,
                             load_model=True,
                             depth=True,
                             model_dir=ckpts[i])
    print('Average Test Reward (with depth buffer):', test_reward)

    test_reward = test_agent(DQN, num_episodes=20,
                             training=False,
                             load_model=True,
                             depth=False,
                             model_dir=ckpts[i])
    print('Average Test Reward (without depth buffer):', test_reward)


Loading model from checkpoints\deadly_corridor.ckpt-420
INFO:tensorflow:Restoring parameters from checkpoints\deadly_corridor.ckpt-420
Test Episode 1 Reward: -50.31547546386719
Test Episode 2 Reward: -104.49089050292969
Test Episode 3 Reward: -104.49089050292969
Test Episode 4 Reward: -104.49089050292969
Test Episode 5 Reward: -11.073715209960938
Test Episode 6 Reward: -104.49089050292969
Test Episode 7 Reward: -52.17103576660156
Test Episode 8 Reward: -5.919708251953125
Test Episode 9 Reward: -104.49089050292969
Test Episode 10 Reward: -105.98927307128906
Test Episode 11 Reward: -108.3341064453125
Test Episode 12 Reward: -104.49089050292969
Test Episode 13 Reward: -114.02394104003906
Test Episode 14 Reward: -115.97746276855469
Test Episode 15 Reward: -104.49089050292969
Test Episode 16 Reward: -104.49089050292969
Test Episode 17 Reward: -102.34445190429688
Test Episode 18 Reward: -104.49089050292969
Test Episode 19 Reward: -104.49089050292969
Test Episode 20 Reward: -104.4908905029296

Test Episode 16 Reward: -110.24053955078125
Test Episode 17 Reward: -41.64593505859375
Test Episode 18 Reward: -53.71795654296875
Test Episode 19 Reward: -110.24053955078125
Test Episode 20 Reward: -98.64634704589844
Average Test Reward (without depth buffer): -72.8001205444336
Loading model from checkpoints\deadly_corridor.ckpt-500
INFO:tensorflow:Restoring parameters from checkpoints\deadly_corridor.ckpt-500
Test Episode 1 Reward: -75.71820068359375
Test Episode 2 Reward: -75.71820068359375
Test Episode 3 Reward: -42.46998596191406
Test Episode 4 Reward: -75.71820068359375
Test Episode 5 Reward: 218.8099365234375
Test Episode 6 Reward: 8.820419311523438
Test Episode 7 Reward: 1.2514801025390625
Test Episode 8 Reward: -75.71820068359375
Test Episode 9 Reward: -33.17030334472656
Test Episode 10 Reward: -8.01446533203125
Test Episode 11 Reward: -56.063262939453125
Test Episode 12 Reward: -1.1046142578125
Test Episode 13 Reward: -115.99250793457031
Test Episode 14 Reward: -111.2575531005

Test Episode 11 Reward: -76.25241088867188
Test Episode 12 Reward: -108.96290588378906
Test Episode 13 Reward: -52.89399719238281
Test Episode 14 Reward: -3.409149169921875
Test Episode 15 Reward: -3.409149169921875
Test Episode 16 Reward: -3.409149169921875
Test Episode 17 Reward: -3.409149169921875
Test Episode 18 Reward: -3.409149169921875
Test Episode 19 Reward: -3.409149169921875
Test Episode 20 Reward: -3.409149169921875
Average Test Reward (without depth buffer): -16.835844421386717
Loading model from checkpoints\deadly_corridor.ckpt-580
INFO:tensorflow:Restoring parameters from checkpoints\deadly_corridor.ckpt-580
Test Episode 1 Reward: -78.36085510253906
Test Episode 2 Reward: -78.36085510253906
Test Episode 3 Reward: 10.797943115234375
Test Episode 4 Reward: -78.36085510253906
Test Episode 5 Reward: -113.39244079589844
Test Episode 6 Reward: 22.413848876953125
Test Episode 7 Reward: -14.540802001953125
Test Episode 8 Reward: -115.93325805664062
Test Episode 9 Reward: -78.3608

Test Episode 5 Reward: -80.00469970703125
Test Episode 6 Reward: 12.20904541015625
Test Episode 7 Reward: 4.9593658447265625
Test Episode 8 Reward: 7.1689300537109375
Test Episode 9 Reward: -31.99688720703125
Test Episode 10 Reward: -80.00469970703125
Test Episode 11 Reward: -28.858154296875
Test Episode 12 Reward: -108.89469909667969
Test Episode 13 Reward: -80.00469970703125
Test Episode 14 Reward: -80.00469970703125
Test Episode 15 Reward: -80.00469970703125
Test Episode 16 Reward: -80.00469970703125
Test Episode 17 Reward: -80.00469970703125
Test Episode 18 Reward: -69.26762390136719
Test Episode 19 Reward: -80.00469970703125
Test Episode 20 Reward: -80.00469970703125
Average Test Reward (without depth buffer): -47.94685440063476
Loading model from checkpoints\deadly_corridor.ckpt-660
INFO:tensorflow:Restoring parameters from checkpoints\deadly_corridor.ckpt-660
Test Episode 1 Reward: -115.97689819335938
Test Episode 2 Reward: -39.63262939453125
Test Episode 3 Reward: -115.97689819

Test Episode 1 Reward: -50.73466491699219
Test Episode 2 Reward: -115.97090148925781
Test Episode 3 Reward: -115.99362182617188
Test Episode 4 Reward: -115.97090148925781
Test Episode 5 Reward: 1.652069091796875
Test Episode 6 Reward: 4.5254974365234375
Test Episode 7 Reward: -50.81666564941406
Test Episode 8 Reward: -104.46783447265625
Test Episode 9 Reward: 78.68965148925781
Test Episode 10 Reward: -115.97090148925781
Test Episode 11 Reward: -50.38909912109375
Test Episode 12 Reward: -115.97090148925781
Test Episode 13 Reward: -20.673110961914062
Test Episode 14 Reward: -31.453216552734375
Test Episode 15 Reward: -115.97090148925781
Test Episode 16 Reward: -91.30203247070312
Test Episode 17 Reward: -115.97090148925781
Test Episode 18 Reward: 276.3733215332031
Test Episode 19 Reward: -38.200042724609375
Test Episode 20 Reward: -115.97090148925781
Average Test Reward (without depth buffer): -50.229302978515626
Loading model from checkpoints\deadly_corridor.ckpt-740
INFO:tensorflow:Rest

Test Episode 19 Reward: 34.233184814453125
Test Episode 20 Reward: -44.0439453125
Average Test Reward (with depth buffer): -26.17293701171875
Loading model from checkpoints\deadly_corridor.ckpt-800
INFO:tensorflow:Restoring parameters from checkpoints\deadly_corridor.ckpt-800
Test Episode 1 Reward: 15.135162353515625
Test Episode 2 Reward: -61.18501281738281
Test Episode 3 Reward: -67.76820373535156
Test Episode 4 Reward: 20.694473266601562
Test Episode 5 Reward: 15.135162353515625
Test Episode 6 Reward: 15.135162353515625
Test Episode 7 Reward: -57.496124267578125
Test Episode 8 Reward: -24.960617065429688
Test Episode 9 Reward: 14.34906005859375
Test Episode 10 Reward: 15.135162353515625
Test Episode 11 Reward: 15.135162353515625
Test Episode 12 Reward: 6.143341064453125
Test Episode 13 Reward: 15.135162353515625
Test Episode 14 Reward: 15.135162353515625
Test Episode 15 Reward: 15.135162353515625
Test Episode 16 Reward: -24.176712036132812
Test Episode 17 Reward: -33.49822998046875
